In [1]:
import numpy as np
import nibabel as nib
from nilearn.image import clean_img, smooth_img, mean_img, index_img, math_img, resample_to_img
from nilearn.plotting import plot_img
from nipype.interfaces.spm import SliceTiming, Coregister, Normalize12, Realign
from nipype.interfaces.spm.utils import ApplyTransform
from nipype import config, logging
from nipype.interfaces import spm

from EDA import *
from dataloaders import *
from utils import *
import config

In [2]:
fmri_path = "D:/Academic/Datasets/Epilepsy_rsfMRI_Fallahi/Sub001/Pre_BOLD/Pre_BOLD.nii.gz"
t1_pre_path = "D:/Academic/Datasets/Epilepsy_rsfMRI_Fallahi/Sub001/Pre_T1MPRAGE/Pre_T1MPRAGE.nii.gz"
t1_post_path = "D:/Academic/Datasets/Epilepsy_rsfMRI_Fallahi/Sub001/Post_T1MPRAGE/Post_T1MPRAGE.nii.gz"

fmri_img = nib.load(fmri_path)
t1_img = nib.load(t1_pre_path)

In [10]:
# Remove the first 10 volumes ---
def remove_initial_volumes(fmri_img, n_volumes=10):

    if isinstance(fmri_img, str):
        fmri_img = nib.load(fmri_img)
    
    # Get the data array
    data = fmri_img.get_fdata()
    
    # Remove the first n volumes
    truncated_data = data[..., n_volumes:]
    
    # Create a new NIfTI image with the truncated data
    out_img = nib.Nifti1Image(truncated_data, affine=fmri_img.affine, header=fmri_img.header)
    
    return out_img

In [12]:
fmri_img_1 = remove_initial_volumes(fmri_path, n_volumes=10)
fmri_img_1.shape

(80, 80, 54, 320)

In [ ]:
# Set the SPM and MATLAB paths
spm.SPMCommand.set_mlab_paths(matlab_cmd='"C:\\Program Files\\MATLAB\\R2022a\\bin\\matlab.exe"', use_mcr=False)


In [38]:
# Motion Correction ---
def motion_correction(fmri_img):

    realign = Realign()
    realign.inputs.in_files = fmri_img
    realign.inputs.register_to_mean = True
    realign.run()

    return realign.outputs.realigned_files

fmri_img_2 = motion_correction(fmri_img_1)

In [6]:
# Slice Timing Correction ---
def slice_timing_correction(fmri_img, tr, slice_order='ascending'):

    stc = SliceTiming()
    stc.inputs.in_files = fmri_img
    stc.inputs.time_repetition = tr
    stc.inputs.slice_order = slice_order
    stc.inputs.ref_slice = int(slice_order[-1] / 2)  # middle slice as reference
    stc.run()

    return stc.outputs.timecorrected_files

# BOLD Signal Filtering ---
def bold_signal_filtering(fmri_img, detrend=True, standardize=True, low_pass=None, high_pass=0.01):

    filtered_img = clean_img(fmri_img, detrend=detrend, standardize=standardize,
                             low_pass=low_pass, high_pass=high_pass)
    return filtered_img

# Global Signal Regression ---
def global_signal_regression(fmri_img):

    gsr_img = clean_img(fmri_img, standardize=False, detrend=False, confounds='global_signal')
    return gsr_img

# Spatial Smoothing ---
def spatial_smoothing(fmri_img, fwhm=6):

    smoothed_img = smooth_img(fmri_img, fwhm=fwhm)
    return smoothed_img

# Coregistration to T1-weighted MRI Scan ---
def coregister_to_mri(fmri_img, t1_img):

    coreg = Coregister()
    coreg.inputs.target = t1_img
    coreg.inputs.source = fmri_img
    coreg.run()

    return coreg.outputs.coregistered_source

# Normalization to MNI Space ---
def normalize_to_mni(coregistered_img, t1_img, template='T1.nii'):

    norm = Normalize12()
    norm.inputs.image_to_align = coregistered_img
    norm.inputs.apply_to_files = coregistered_img
    norm.inputs.jobtype = 'estwrite'
    norm.run()

    return norm.outputs.normalized_files

# Framewise_Displacement (FD) ---
def calculate_framewise_displacement(motion_params_file, radius=50.0):
    """
    Calculate framewise displacement (FD) from motion parameters.
    
    Parameters:
    - motion_params_file: Path to a text file containing motion correction parameters.
                          Each row should represent a volume, and the columns should be:
                          [translation_x, translation_y, translation_z, rotation_x, rotation_y, rotation_z]
    - radius: The radius of the sphere used to convert rotational displacements into linear displacements.
    
    Returns:
    - fd: An array containing the framewise displacement for each volume.
    """
    # Load motion parameters (6 parameters per volume)
    motion_params = np.loadtxt(motion_params_file)
    
    # Calculate the difference between successive timepoints (volumes)
    diff = np.diff(motion_params, axis=0)
    
    # Convert rotational displacements to mm
    rotations = diff[:, 3:] * radius
    
    # Combine translations and rotations
    diff[:, 3:] = rotations
    
    # Framewise displacement is the sum of the absolute differences for each volume
    fd = np.sum(np.abs(diff), axis=1)
    
    # Insert a 0 at the beginning since the first volume has no preceding volume
    fd = np.insert(fd, 0, 0)
    
    return fd

# Scrubbing (outlier removal) ---
def scrub_volumes(fmri_img, fd, fd_threshold=0.5):
    """
    Remove volumes with excessive motion (scrubbing) based on a framewise displacement threshold.
    
    Parameters:
    - fmri_img: Nifti image of fMRI data.
    - fd: Array of framewise displacement values.
    - fd_threshold: Framewise displacement threshold for scrubbing.
    
    Returns:
    - scrubbed_img: Nifti image with volumes exceeding the FD threshold removed.
    """
    # This is a simplified example; you'll need FD calculation separately
    fd = calculate_framewise_displacement(fmri_img)
    
    # Identify indices of volumes with FD below the threshold
    keep_indices = np.where(fd < fd_threshold)[0]
    
    # Retain only the volumes that meet the FD criteria
    scrubbed_img = index_img(fmri_img, keep_indices)
    
    return scrubbed_img

# Temporal masking ---
def apply_mask(fmri_img, mask_img):

    masked_img = math_img("img1 * img2", img1=fmri_img, img2=mask_img)
    
    return masked_img

# Save nifti ---
def save_fmri(fmri_img, output_path):
    
    # Save the NIfTI image to the specified path
    nib.save(fmri_img, output_path)
    
    print(f"fMRI data saved to: {output_path}")

In [12]:
def preprocess(fmri_path, t1_path, tr, fwhm=6):
   
    # Load fMRI data
    fmri_img = nib.load(fmri_path)
    t1_img = nib.load(t1_path)

    # Step 1: Remove the first 10 volumes
    fmri_img = remove_initial_volumes(fmri_path, n_volumes=10)

    # Step 2: Motion Correction
    fmri_img = motion_correction(fmri_img)

    # # Step 3: Slice Timing Correction
    # fmri_img = slice_timing_correction(fmri_img, tr=tr)

    # # Optional: Artifact Removal using ICA-AROMA (if FSL is available)
    # # fmri_img = ica_aroma(fmri_img)

    # # Step 4: BOLD Signal Filtering
    # fmri_img = bold_signal_filtering(fmri_img)

    # # Step 5: Global Signal Regression (optional)
    # # fmri_img = global_signal_regression(fmri_img)

    # # Step 6: Spatial Smoothing
    # fmri_img = spatial_smoothing(fmri_img, fwhm=fwhm)

    # # Step 7: Coregistration to MRI Scan
    # fmri_img = coregister_to_mri(fmri_img, t1_img)

    # # Step 8: Normalization to MNI Space
    # fmri_img = normalize_to_mni(fmri_img, t1_img)

    # Optional: Scrubbing (outlier removal)
    # fmri_img = scrub_volumes(fmri_img)

    # Optionally visualize the final preprocessed image
    plot_img(mean_img(fmri_img), title="Final Preprocessed fMRI Image")

    return fmri_img

In [ ]:
fmri_file = 'D:\Academic\Datasets\Epilepsy_rsfMRI_Fallahi\Sub001\Pre_BOLD\Pre_BOLD.nii.gz'
t1_file = 'D:\Academic\Datasets\Epilepsy_rsfMRI_Fallahi\Sub001\Pre_T1MPRAGE\Pre_T1MPRAGE.nii.gz'
output_file = 'D:\Academic\Datasets\Epilepsy_rsfMRI_Fallahi\Sub001\Pre_BOLD\Pre_BOLD_preprocessed.nii.gz'
tr = 3.0  # TR in seconds

preprocessed_fmri = preprocess(fmri_file, t1_file, tr)
save_fmri(preprocessed_fmri, output_file)

In [ ]:
import os
import nipype.interfaces.fsl as fsl
import nipype.interfaces.io as nio
from nipype import Node, Workflow
from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.fsl.maths import TemporalFilter
import platform

if platform.system() == 'Linux':
    import pwd

# Set the FSL output type to NIfTI
fsl.FSLCommand.set_default_output_type('NIFTI_GZ')

# Define the preprocessing workflow
def create_preprocessing_workflow(fmri_file, output_dir, tr):
    # Workflow and nodes
    workflow = Workflow(name='preprocessing')
    workflow.base_dir = output_dir

    # Identity Interface to pass data around
    infosource = Node(IdentityInterface(fields=['fmri']), name='infosource')
    infosource.inputs.fmri = fmri_file

    # Remove initial volumes
    def remove_volumes(in_file, num_volumes):
        import nibabel as nib
        img = nib.load(in_file)
        img_data = img.get_fdata()[..., num_volumes:]
        img_header = img.header
        img_affine = img.affine
        new_img = nib.Nifti1Image(img_data, img_affine, img_header)
        out_file = os.path.abspath('fmri_no_initial_volumes.nii.gz')
        nib.save(new_img, out_file)
        return out_file

    remove_volumes_node = Node(name='remove_volumes', interface=nio.Function(
        input_names=['in_file', 'num_volumes'],
        output_names=['out_file'],
        function=remove_volumes))
    remove_volumes_node.inputs.num_volumes = 10

    # Slice Timing Correction
    slicetiming = Node(fsl.SliceTimer(time_repetition=tr), name='slicetiming')

    # Motion Correction
    motion_correction = Node(fsl.MCFLIRT(save_plots=True), name='motion_correction')

    # Temporal Filtering (High-pass filter)
    temp_filter = Node(TemporalFilter(highpass_sigma=50), name='temp_filter')

    # Spatial Smoothing
    smoothing = Node(fsl.IsotropicSmooth(fwhm=6), name='smoothing')

    # Coregistration (Functional to Anatomical)
    # Assuming you have an anatomical image to coregister to
    coregistration = Node(fsl.FLIRT(dof=6, cost='corratio'), name='coregistration')
    coregistration.inputs.reference = t1_pre_path


    # Normalization to MNI Space
    normalization = Node(fsl.FLIRT(reference=fsl.Info.standard_image('MNI152_T1_2mm_brain')), name='normalization')

    # Save preprocessed data
    datasink = Node(nio.DataSink(base_directory=output_dir), name='datasink')

    # Connect the nodes in the workflow
    workflow.connect([
        (infosource, remove_volumes_node, [('fmri', 'in_file')]),
        (remove_volumes_node, slicetiming, [('out_file', 'in_file')]),
        (slicetiming, motion_correction, [('slice_time_corrected_file', 'in_file')]),
        (motion_correction, temp_filter, [('out_file', 'in_file')]),
        (temp_filter, smoothing, [('out_file', 'in_file')]),
        (smoothing, coregistration, [('out_file', 'in_file')]),
        (coregistration, normalization, [('out_file', 'in_file')]),
        (normalization, datasink, [('out_file', '@normalized')])
    ])

    return workflow

preproc_workflow = create_preprocessing_workflow(fmri_file, output_dir, tr)
preproc_workflow.run('MultiProc', plugin_args={'n_procs': 4})
